## 1. Carga de datos
#### Elimino variables no relevantes para caso estudio ya que no voy a hacer procesamiento de texto (Url's, Id's, descripciones y metadata).


In [4]:
import numpy  as np  
import pandas as pd

In [7]:
raw_data = pd.read_csv("airbnb-listings-extract.csv", sep=';')  # cargamos fichero
print(raw_data.shape)
raw_data.head(5).T  

(14780, 89)


,0,1,2,3,4
ID,11210388,17471131,17584891,5398030,18104606
Listing Url,https://www.airbnb.com/rooms/11210388,https://www.airbnb.com/rooms/17471131,https://www.airbnb.com/rooms/17584891,https://www.airbnb.com/rooms/5398030,https://www.airbnb.com/rooms/18104606
Scrape ID,20170306202425,20170407214050,20170407214050,20170407214050,20170407214050
Last Scraped,2017-03-07,2017-04-08,2017-04-08,2017-04-08,2017-04-08
Name,The Loft-Full Bath-Deck w/View,"Claris I, Friendly Rentals","Style Terrace Red, Friendly Rentals",Picasso Suite 1.4 Paseo de Gracia,Smart City Centre Apartment II
...,...,...,...,...,...
Cancellation Policy,moderate,super_strict_30,super_strict_30,strict,flexible
Calculated host listings count,1.0,106.0,106.0,24.0,92.0
Reviews per Month,3.5,0.86,NaN,1.09,NaN
Geolocation,"30.3373609355,-97.8632766782","41.3896829422,2.17262543017","41.3930345489,2.16217327868","41.3969668101,2.1674178103","41.3886851936,2.15514963616"


In [13]:
# Análisis específico de Country = Spain
print("\nANÁLISIS ESPECÍFICO DE SPAIN:")
print("=" * 40)

if 'Country' in raw_data.columns:
    total_records = len(raw_data)
    spain_records = len(raw_data[raw_data['Country Code'] == 'ES'])
    spain_percentage = (spain_records / total_records) * 100
    
    print(f"Total de registros en el dataset: {total_records:,}")
    print(f"Registros de España: {spain_records:,}")
    print(f"Porcentaje de España: {spain_percentage:.2f}%")
    
    # Mostrar también otros países para contexto
    print(f"\nDistribución por países (top 10):")
    country_counts = raw_data['Country'].value_counts().head(10)
    for country, count in country_counts.items():
        percentage = (count / total_records) * 100
        print(f"  {country:20s}: {count:6,} ({percentage:5.2f}%)")
else:
    print("La columna 'Country' no se encuentra en el dataset")


ANÁLISIS ESPECÍFICO DE SPAIN:
Total de registros en el dataset: 14,780
Registros de España: 14,001
Porcentaje de España: 94.73%

Distribución por países (top 10):
  Spain               : 14,001 (94.73%)
  United States       :    184 ( 1.24%)
  United Kingdom      :    159 ( 1.08%)
  France              :    110 ( 0.74%)
  Italy               :     68 ( 0.46%)
  Australia           :     46 ( 0.31%)
  Ireland             :     45 ( 0.30%)
  Germany             :     41 ( 0.28%)
  Austria             :     31 ( 0.21%)
  Netherlands         :     26 ( 0.18%)


In [9]:
raw_data[existing_columns].head(5).T

,0,1,2,3,4
ID,11210388,17471131,17584891,5398030,18104606
Listing Url,https://www.airbnb.com/rooms/11210388,https://www.airbnb.com/rooms/17471131,https://www.airbnb.com/rooms/17584891,https://www.airbnb.com/rooms/5398030,https://www.airbnb.com/rooms/18104606
Scrape ID,20170306202425,20170407214050,20170407214050,20170407214050,20170407214050
Thumbnail Url,https://a0.muscache.com/im/pictures/c351a9b5-5...,https://a0.muscache.com/im/pictures/3ce27aee-f...,https://a0.muscache.com/im/pictures/fd4528f3-f...,https://a0.muscache.com/im/pictures/77343485/3...,https://a0.muscache.com/im/pictures/d26f6f1b-f...
Medium Url,https://a0.muscache.com/im/pictures/c351a9b5-5...,https://a0.muscache.com/im/pictures/3ce27aee-f...,https://a0.muscache.com/im/pictures/fd4528f3-f...,https://a0.muscache.com/im/pictures/77343485/3...,https://a0.muscache.com/im/pictures/d26f6f1b-f...
Picture Url,https://public.opendatasoft.com/api/v2/catalog...,https://public.opendatasoft.com/api/v2/catalog...,https://public.opendatasoft.com/api/v2/catalog...,https://public.opendatasoft.com/api/v2/catalog...,https://public.opendatasoft.com/api/v2/catalog...
XL Picture Url,https://a0.muscache.com/im/pictures/c351a9b5-5...,https://a0.muscache.com/im/pictures/3ce27aee-f...,https://a0.muscache.com/im/pictures/fd4528f3-f...,https://a0.muscache.com/im/pictures/77343485/3...,https://a0.muscache.com/im/pictures/d26f6f1b-f...
Host URL,https://www.airbnb.com/users/show/58386783,https://www.airbnb.com/users/show/102947901,https://www.airbnb.com/users/show/102947901,https://www.airbnb.com/users/show/131794,https://www.airbnb.com/users/show/54310958
Host Thumbnail Url,https://a0.muscache.com/im/pictures/c5c78f08-f...,https://a0.muscache.com/im/pictures/c071e13e-5...,https://a0.muscache.com/im/pictures/c071e13e-5...,https://a0.muscache.com/im/pictures/a79b0ef0-f...,https://a0.muscache.com/im/pictures/8f3ec0f2-1...
Host Picture Url,https://a0.muscache.com/im/pictures/c5c78f08-f...,https://a0.muscache.com/im/pictures/c071e13e-5...,https://a0.muscache.com/im/pictures/c071e13e-5...,https://a0.muscache.com/im/pictures/a79b0ef0-f...,https://a0.muscache.com/im/pictures/8f3ec0f2-1...


In [ ]:
# Crear una copia de los datos para trabajar y eliminar las variables 
input_data = raw_data.copy()

columns_to_drop = [
    'ID', 'Listing Url', 'Scrape ID', 'Thumbnail Url', 'Medium Url', 
    'Picture Url', 'XL Picture Url', 'Host URL', 'Host Thumbnail Url', 
    'Host Picture Url', 'Last Scraped', 'Name', 'Summary', 'Space', 
    'Description', 'Neighborhood Overview', 'License', 'Jurisdiction Names', 
    'Cancellation Policy', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules', 'Host ID', 'Host Name', 
    'Host Location', 'Host About', 'Host Response Time', 'Geolocation', 'Smart Location', 'Market', 'Street', 
    'Host Verifications', 'Neighbourhood', 'Neighbourhood Cleansed', 
    'Neighbourhood Group Cleansed', 'Country Code', 'Amenities', 'Calendar Updated', 
    'Calendar last Scraped', 'Features', 'Host Neighbourhood'
]

selected_data = input_data.drop(columns=columns_to_drop)  # APLICANDO la eliminación de columnas

print(f"Dataset original: {raw_data.shape}")
print(f"Dataset después de limpieza: {selected_data.shape}")
selected_data.head(5).T

## 2. División de datos Train / Test
#### Después aplico método `train_test_split` para separación fisica en dos archivos .csv (datos de entrenamiento y prueba)

## 3. Análisis exploratorio
#### Miro los datos de Train para saber distribuciones estadisticas, outliers, correlaciones.
- Head, describe...
- Histogramas, scatterplots..
- Matriz correlaciones entre variables
- Método de filtrado: `f_reg, mutual_info_reg`

## 4. Preprocesado datos Train
#### Proceso datos de train en función de las conclusiones estadisticas
- Eliminación por random forest/Lasso, muchos Nan, alta correlación..
- Imputación de valores ausentes
- Codificación variables categoricas
- Generación y transformación de variables
- Filtrado outliers

## 5. Procesado datos Test
#### Aplico las mismas transformaciones que en Train en el punto 5

## 6. Preparacion dataset Train / Test
#### Normalizar Test según Train si aplica

## 7. Modelado
#### Cross validation

## 8. Evaluación